# Utilidades de manipulación de imágenes y plantilla para arquitecturas de prueba

## 1. Manipulado de imágenes

Las siguiente funciones nos permiten recortar imágenes, de modo que se obtienen imágenes cuadradas, y generar conjuntos de entrenamiento y prueba que se almacenan en disco en una estructura de directorios que permiten a Keras interpretar a cada imagen en el tipo de conjunto al que pertenece (entrenamiento/prueba) y en la categoría o clase con la que se corresponde.

In [1]:
%matplotlib inline

from itertools import groupby
from random import shuffle
import os
from keras.preprocessing.image import load_img, list_pictures
import shutil



def cut_up_picture(p):
    """
    Dada una imagen, devuelve los dos trozos 
    cuadrados más extremos de la misma.
    """
    l = min(p.size)
    p1 = p.crop((0, 0, l, l))
    p2 = p.crop((p.width - l, p.height - l, p.width, p.height))
    return p1, p2
    
def fold(pictures):
    """
    Agrupa las imágenes originales en conjuntos
    train y validation, cada una en su categoría 
    dentro de cada conjunto.
    La selección la realiza de forma aleatoria, 
    según un 80%, 20%
    """
    groups = groupby(pictures, lambda x: x.split("/")[-1].split("_")[0])
    result = {'train':{},'validation':{}}
    for k, g in groups:
        g = list(g)
        shuffle(g)
        a = int(round(len(g) * 0.8))
        result['train'][k] = g[:a]
        result['validation'][k] = g[a:]
    return result

def create_data(folds):
    """
    Crea los conjuntos de entrenamiento y validación
    con imágenes cuadradas organizadas en directorios
    directamente utilizables por Keras.
    Las guarda en el directorio data.
    """
    if not os.path.exists('data'):
        os.makedirs('data') 
    for k0,v0 in folds.iteritems():
        if not os.path.exists('data/{}'.format(k0)):
            os.makedirs('data/{}'.format(k0)) 
        for k1,v1 in v0.iteritems():
            if not os.path.exists('data/{}/{}'.format(k0,k1)):
                os.makedirs('data/{}/{}'.format(k0,k1))
            for path in v1:
                name = path.split("/")[-1]
                p0,p1 = cut_up_picture(load_img(path))
                p0.save('data/{}/{}/0_{}'.format(k0,k1,name))
                p0.save('data/{}/{}/1_{}'.format(k0,k1,name))

def regenerate_all_data():
    shutil.rmtree('data/train', ignore_errors=True)
    os.makedirs('data/train')
    shutil.rmtree('data/validation', ignore_errors=True)
    os.makedirs('data/validation')
    pictures = list_pictures('imagenesdepolen')
    folds = fold(pictures)
    create_data(folds)
    

# Sólo es necesario hacerlo una vez    
# regenerate_all_data()

Using TensorFlow backend.


## 2. Generadores de imágenes

Una vez tenemos las imágenes en disco organizadas de una forma que Keras pueda interpretarlas automáticamente, según se ha descrito en el punto anterior, podeos definir los generados que alimenten el proceso de entrenamiento y prueba de la red automáticamente.

Teniendo las imágenes organizadas según el punto anterior, y utilizando estos generadores, podemos centrarnos en las arquitecturas y no tener que dedicar más esfuerzo a la parte de la entrada de la red.

In [2]:
from keras.preprocessing.image import ImageDataGenerator


# dimensions of our images.
img_width, img_height = 100, 100
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
#     save_to_dir='preview/train',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale')

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
#     save_to_dir='preview/validation',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale')

# Utilidad de visualización
# Sólo si se activan los parámetros save_to_dir en los generadores

# [os.remove('preview/train/' + f) for f in os.listdir('preview/train')]
# [os.remove('preview/validation/' + f) for f in os.listdir('preview/validation')]
# for batch in train_generator:
#     break
# for batch in validation_generator:
#     break

Found 1152 images belonging to 21 classes.
Found 288 images belonging to 21 classes.


## 3. Plantilla para arquitecturas con Keras

Básicamente aquí se muestra un ejemplo de CNN diseñado con Keras y haciendo uso de los generadores anteriores.

In [3]:

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


train_samples = 640
validation_samples = 320

# Pongo sólo 2 epochs simplemente para probar que todo
# funciona bien. Cuando se intente entrenar una red de 
# verdad esto tendrá que ser más alto, quizá 50
epochs = 2


model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
 

### 3.1. Entrenamiento sin monitorizar con TensorBoard

In [4]:
model.fit_generator(
    train_generator,
    samples_per_epoch=train_samples,
    nb_epoch=epochs,
    validation_data=validation_generator,
    nb_val_samples=validation_samples,
    verbose=2)

Epoch 1/2
10s - loss: -1.2329e+02 - acc: 0.0531 - val_loss: -1.4687e+02 - val_acc: 0.0437
Epoch 2/2
9s - loss: -1.5038e+02 - acc: 0.0406 - val_loss: -1.4642e+02 - val_acc: 0.0469


### 3.2. Entrenamiento monitorizando con TensorBoard

En el caso de que se utilice TensorFlow como backend contra el que trabaja Keras, se puede monitorizar la evolución del entrenamiento de la red con TensorBoard.

In [5]:
from keras.callbacks import TensorBoard

tb = TensorBoard(
    log_dir='./logs', 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False)

model.fit_generator(
    train_generator,
    samples_per_epoch=train_samples,
    nb_epoch=epochs,
    validation_data=validation_generator,
    nb_val_samples=validation_samples,
    verbose=2,
    callbacks=[tb])

Epoch 1/2
9s - loss: -1.4904e+02 - acc: 0.0516 - val_loss: -1.4986e+02 - val_acc: 0.0469
Epoch 2/2
8s - loss: -1.4508e+02 - acc: 0.0484 - val_loss: -1.4732e+02 - val_acc: 0.0469


### 3.3. Y evaluación del resultado

In [6]:
score = model.evaluate_generator(validation_generator, 50)
print('Test score:', score[0])
print('Test accuracy:', score[1])

('Test score:', -141.98686218261719)
('Test accuracy:', 0.078125)
